# Learning to Classify Text

## Supervised Classification

Classification is the task of choosing the correct class label for a given input. A classifier is called supervised if it is built based on training corpora containing the correct label for each input.

### Gender Identification

Let's build a classifier to determine whether a given name is of a male or a female person. To start with, we'll build a function that extracts the final letter of a given word and returns a *feature dictionary*.

In [1]:
def gender_features(word):
    return {'last_letter': word[-1]}

In [2]:
gender_features('Shrek')

{'last_letter': 'k'}

The key in the dictionary is the feature name. Let's prepare a list of examples of names and corresponding class labels (males or females)

In [2]:
import nltk
from nltk.corpus import names
labeled_names = (
    [(name, 'male') for name in names.words('male.txt')] + [(name, 'female') for name in names.words('female.txt')])

In [9]:
import random
random.shuffle(labeled_names)

We can now use the feature extractor to process the data, and divide it into train and test sets.

In [10]:
featuresets = [(gender_features(n), gender) for (n, gender) in labeled_names]
train_set, test_set = featuresets[500:], featuresets[:500]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [11]:
# Testing the classifier on names that do not appear in the training set
print('Neo:', classifier.classify(gender_features('Neo')))
print('Trinity:', classifier.classify(gender_features('Trinity')))

Neo: male
Trinity: female


In [12]:
# Evaulating the classifier on the test set
print(nltk.classify.accuracy(classifier, test_set))

0.786


We can examine the classifier to determine the most effective features:

In [13]:
classifier.show_most_informative_features(5)

Most Informative Features
             last_letter = 'a'            female : male   =     37.3 : 1.0
             last_letter = 'k'              male : female =     30.8 : 1.0
             last_letter = 'f'              male : female =     17.1 : 1.0
             last_letter = 'p'              male : female =     12.4 : 1.0
             last_letter = 'v'              male : female =      9.7 : 1.0


According to the above the names in the training set that end in "a" are female 33 times more often than they are male.

When working with a large corpora, constructing a list for features can quickly use up a large amount of memory. In these cases, we can use `nltk.classify.apply_features` which returns an object that acts like a list but does not store the feature sets in memory.

In [18]:
from nltk.classify import apply_features
train_set = apply_features(gender_features, labeled_names[500:])
test_set = apply_features(gender_features, labeled_names[:500])

### Choosing the Right Features

Let's improve upon our feature extractor for gender names by adding more features.

In [19]:
def gender_features2(name):
    features = {}
    features["first_letter"] = name[0].lower()
    features["last_letter"] = name[-1].lower()
    for letter in "abcdefghijklmnopqrstuvwxyz":
        features["count({})".format(letter)] = name.lower().count(letter)
        features["has({})".format(letter)] = (letter in name.lower())
    return features

In [22]:
print(gender_features2('John'))

{'first_letter': 'j', 'last_letter': 'n', 'count(a)': 0, 'has(a)': False, 'count(b)': 0, 'has(b)': False, 'count(c)': 0, 'has(c)': False, 'count(d)': 0, 'has(d)': False, 'count(e)': 0, 'has(e)': False, 'count(f)': 0, 'has(f)': False, 'count(g)': 0, 'has(g)': False, 'count(h)': 1, 'has(h)': True, 'count(i)': 0, 'has(i)': False, 'count(j)': 1, 'has(j)': True, 'count(k)': 0, 'has(k)': False, 'count(l)': 0, 'has(l)': False, 'count(m)': 0, 'has(m)': False, 'count(n)': 1, 'has(n)': True, 'count(o)': 1, 'has(o)': True, 'count(p)': 0, 'has(p)': False, 'count(q)': 0, 'has(q)': False, 'count(r)': 0, 'has(r)': False, 'count(s)': 0, 'has(s)': False, 'count(t)': 0, 'has(t)': False, 'count(u)': 0, 'has(u)': False, 'count(v)': 0, 'has(v)': False, 'count(w)': 0, 'has(w)': False, 'count(x)': 0, 'has(x)': False, 'count(y)': 0, 'has(y)': False, 'count(z)': 0, 'has(z)': False}


We have to keep in mind that providing too many features to the model can lead to overfitting on the training data. Thus, the model will not be able to generalize well to unseen data. 

Let's train our model with the new features and see how it performs on the test set.

In [23]:
featuresets = [(gender_features2(n), gender) for (n, gender) in labeled_names]
train_set, test_set = featuresets[500:], featuresets[:500]

In [24]:
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.782


As we can see above, the new model using the new featureset does not offer much improvement over the model that uses just the last letter of the name as a feature. In fact, it performs *worse*. 

An efficient method for refining the feature set is *error analysis*. First, we select a development set, containing the corpus data for creating the model. This development set is then subdivided into the training set and the dev-test set.

We'll follow this approach now.

In [25]:
train_names = labeled_names[1500:]
devtest_names = labeled_names[500:1500]
test_names = labeled_names[:500]

The training set is used to train the model, and the dev-test set is used to perform error analysis. The test set serves in our final evaluation of the system.

Now that we have divided the corpus into appropriate datasets, we'll train our model using the *train* set and test it on the *dev-test* set.

In [26]:
train_set = [(gender_features(n), gender) for (n, gender) in train_names]
devtest_set = [(gender_features(n), gender) for (n, gender) in devtest_names]
test_set = [(gender_features(n), gender) for (n, gender) in test_names]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, devtest_set))

0.76


Using the dev-test set, we can check the errors that the classifier makes when predicting the name genders. 

In [27]:
errors = []
for (name, tag) in devtest_names:
    guess = classifier.classify(gender_features(name))
    if guess != tag:
        errors.append((tag, guess, name))

In [30]:
for (tag, guess, name) in sorted(errors)[:20]:
    print('correct={:<8} guess={:<8s} name={:<30}'.format(tag, guess, name))

correct=female   guess=male     name=Allison                       
correct=female   guess=male     name=Allsun                        
correct=female   guess=male     name=Amargo                        
correct=female   guess=male     name=Ardeen                        
correct=female   guess=male     name=Avrit                         
correct=female   guess=male     name=Bill                          
correct=female   guess=male     name=Brenn                         
correct=female   guess=male     name=Brittan                       
correct=female   guess=male     name=Carleen                       
correct=female   guess=male     name=Carlen                        
correct=female   guess=male     name=Carolann                      
correct=female   guess=male     name=Caryn                         
correct=female   guess=male     name=Cass                          
correct=female   guess=male     name=Cathyleen                     
correct=female   guess=male     name=Catlin     

If we analyse the list above, we will notice that the *suffixes* seem to be important for classifying a name gender i.e. we have to look at more than one letter at the end of the names.

We can, thus, accordingly modify the feature extractor to include the same.

In [31]:
def gender_features(word):
    return {
        'suffix1': word[-1:],
        'suffix2': word[-2:]}

In [32]:
train_set = [(gender_features(n), gender) for (n, gender) in train_names]
devtest_set = [(gender_features(n), gender) for (n, gender) in devtest_names]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, devtest_set))

0.794


We can see the the performance of our model has improved by almost 1%, compared to the one that takes a look only at the last letter.

### Document Classification

Let's take a look at a case of classifying documents (movies) into categories (positive or negative).

In [36]:
from nltk.corpus import movie_reviews
documents = [
    (list(movie_reviews.words(fileid)), category)
    for category in movie_reviews.categories()
    for fileid in movie_reviews.fileids(category)]

In [43]:
print(documents[0][0][:40])

['plot', ':', 'two', 'teen', 'couples', 'go', 'to', 'a', 'church', 'party', ',', 'drink', 'and', 'then', 'drive', '.', 'they', 'get', 'into', 'an', 'accident', '.', 'one', 'of', 'the', 'guys', 'dies', ',', 'but', 'his', 'girlfriend', 'continues', 'to', 'see', 'him', 'in', 'her', 'life', ',', 'and']


In [44]:
random.shuffle(documents)

Now let's define a feature extractor for documents. We can define a feature for each word, indicating whether or not a document containts that word. 

To limit the number of features that the classifier will need to process, we can construct a list of the most frequent 200 words, and define a feature extractor only for those words.

In [45]:
all_words = nltk.FreqDist(w.lower() for w in movie_reviews.words())
word_features = list(all_words)[:2000]

In [46]:
def document_features(document):
    document_words = set(document)
    features = {"contains({})".format(word): (word in document_words) for word in word_features}
    return features

In [49]:
# Uncomment the following line to check the results of feature extraction on one document
# print(document_features(movie_reviews.words('pos/cv957_8737.txt')))

We can now train a classifier to label new movie reviews.

In [50]:
featuresets = [(document_features(d), c) for (d,c) in documents]
train_set, test_set = featuresets[100:], featuresets[:100]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [51]:
print(nltk.classify.accuracy(classifier, test_set))

0.82


In [52]:
classifier.show_most_informative_features(5)

Most Informative Features
 contains(unimaginative) = True              neg : pos    =      7.6 : 1.0
    contains(schumacher) = True              neg : pos    =      7.4 : 1.0
     contains(atrocious) = True              neg : pos    =      7.0 : 1.0
        contains(suvari) = True              neg : pos    =      7.0 : 1.0
        contains(shoddy) = True              neg : pos    =      7.0 : 1.0


### Part-of-Speech Tagging

For tagging parts of speech, we can train a classifier to work out which suffixes are most informative. 

First, let's check out which suffixes are most common.

In [119]:
from nltk.corpus import brown
suffix_fdist = nltk.FreqDist()
for word in brown.words():
    word = word.lower()
    suffix_fdist[word[-1:]] += 1
    suffix_fdist[word[-2:]] += 1
    suffix_fdist[word[-3:]] += 1

In [120]:
common_suffixes = [suffix for (suffix, count) in suffix_fdist.most_common(100)]
print(common_suffixes)

['e', ',', '.', 's', 'd', 't', 'he', 'n', 'a', 'of', 'the', 'y', 'r', 'to', 'in', 'f', 'o', 'ed', 'nd', 'is', 'on', 'l', 'g', 'and', 'ng', 'er', 'as', 'ing', 'h', 'at', 'es', 'or', 're', 'it', '``', 'an', "''", 'm', ';', 'i', 'ly', 'ion', 'en', 'al', '?', 'nt', 'be', 'hat', 'st', 'his', 'th', 'll', 'le', 'ce', 'by', 'ts', 'me', 've', "'", 'se', 'ut', 'was', 'for', 'ent', 'ch', 'k', 'w', 'ld', '`', 'rs', 'ted', 'ere', 'her', 'ne', 'ns', 'ith', 'ad', 'ry', ')', '(', 'te', '--', 'ay', 'ty', 'ot', 'p', 'nce', "'s", 'ter', 'om', 'ss', ':', 'we', 'are', 'c', 'ers', 'uld', 'had', 'so', 'ey']


Let's define a feature extractor which checks a given word for these suffixes, and train a decision tree classifier. 

In [121]:
def pos_features(word):
    return {"endswith({})".format(suffix): word.lower().endswith(suffix) for suffix in common_suffixes}

In [122]:
tagged_words = brown.tagged_words(categories='news')
featuresets = [(pos_features(n), g) for (n,g) in tagged_words]

In [123]:
size = int(len(featuresets)*0.1)
train_set_tags, test_set_tags = featuresets[size:], featuresets[:size]

In [125]:
classifier = nltk.DecisionTreeClassifier.train(train_set_tags)

In [126]:
nltk.classify.accuracy(classifier, test_set_tags)

0.6270512182993535

In [127]:
classifier.classify(pos_features('cats'))

'NNS'

We can use NLTK to print out the decision tree model as pseudocode:

In [128]:
print(classifier.pseudocode(depth=4))

if endswith(the) == False: 
  if endswith(,) == False: 
    if endswith(s) == False: 
      if endswith(.) == False: return '.'
      if endswith(.) == True: return '.'
    if endswith(s) == True: 
      if endswith(is) == False: return 'PP$'
      if endswith(is) == True: return 'BEZ'
  if endswith(,) == True: return ','
if endswith(the) == True: return 'AT'



### Exploiting Context

Let's modify our feature extractor to use an untagged sentence so that we can train a classifier to tag words with part-of-speech while looking at the context.

In [129]:
def pos_features(sentence, i):
    features = {
        "suffix(1)": sentence[i][-1:],
        "suffix(2)": sentence[i][-2:],
        "suffix(3)": sentence[i][-3:]}
    if i==0:
        features["prev-word"] = "<START>"
    else:
        features["prev-word"] = sentence[i-1]
    return features        

In [130]:
# testing the feature extractor
pos_features(brown.sents()[0], 8)

{'suffix(1)': 'n', 'suffix(2)': 'on', 'suffix(3)': 'ion', 'prev-word': 'an'}

In [131]:
tagged_sents = brown.tagged_sents(categories='news')
featuresets = []
for tagged_sent in tagged_sents:
    untagged_sent = nltk.tag.untag(tagged_sent)
    for i, (word, tag) in enumerate(tagged_sent):
        featuresets.append((pos_features(untagged_sent, i), tag))

In [132]:
size = int(len(featuresets)*0.1)
train_set, test_set = featuresets[size:], featuresets[:size]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [133]:
nltk.classify.accuracy(classifier, test_set)

0.7891596220785678

### Sequence Classification

The goal of classifying a sequence is to find the most appropriate labellings for a collection of related inputs. 

One strategy for sequence classification is known as *consecutive classification* which infs the most ikely class label for the first input and then use that answer to help find the best label for the next input. 

First, we must modify our pos feature extractor to take a `history` argument, which will provide a list of tags that we've predicted for the sentence so far.

In [7]:
def pos_features(sentence, i, history):
    features = {
        "suffix(1)": sentence[i][-1:],
        "suffix(2)": sentence[i][-2:],
        "suffix(3)": sentence[i][-3:]}
    if i==0:
        features["prev-word"] = "<START>"
        features["prev-tag"] = "<START>"
    else:
        features["prev-word"] = sentence[i-1]
        features["prev-tag"] = history[i-1]
    return features

Let's build our sequence classifier now. 

In [3]:
class ConsecutivePosTagger(nltk.TaggerI):
    
    def __init__(self, train_sents):
        train_set = []
        for tagged_sent in train_sents:
            untagged_sent = nltk.tag.untag(tagged_sent)
            history = []
            for i, (word, tag) in enumerate(tagged_sent):
                featureset = pos_features(untagged_sent , i, history)
                train_set.append((featureset, tag))
                history.append(tag)
        self.classifier = nltk.NaiveBayesClassifier.train(train_set)
    
    def tag(self, sentence):
        history = []
        for i, word in enumerate(sentence):
            featureset = pos_features(sentence, i, history)
            tag = self.classifier.classify(featureset)
            history.append(tag)
        return zip(sentence, history)

In [8]:
tagged_sents = brown.tagged_sents(categories='news')
size = int(len(tagged_sents) * 0.1)
train_sents, test_sents = tagged_sents[size:], tagged_sents[:size]
tagger = ConsecutivePosTagger(train_sents)
print(tagger.evaluate(test_sents))

0.7980528511821975


## Further Examples of Supervised Classification

### Sentence Segmentation

Sentence segmentation can be viewed as a classification task for punctuation: whenever we encounter a symbol that could possibly end a sentence, such as a period or a question mark, we have to decide whether it terminates the preceding sentence.

First, let's obtain data that is already segmented into sentences and wrangle it into the correct form.

In [12]:
sents = nltk.corpus.treebank_raw.sents()
# tokens: merged list of tokens from individual sentences
tokens = []
# boundaries: set of indices of all sentence-boundary tokens
boundaries = set()
offset = 0
for sent in sents:
    tokens.extend(sent)
    offset += len(sent)
    boundaries.add(offset-1)

Now, let's write a feature extractor for this classification task.

In [18]:
def punct_features(tokens, i):
    return {
        'next-word-capitalized': tokens[i+1][0].isupper(),
        'prev-word': tokens[i-1].lower(),
        'punct': tokens[i],
        'prev-word-is-one-char': len(tokens[i-1]) == 1
    }

In [19]:
featuresets = [(punct_features(tokens, i), (i in boundaries))
               for i in range(1, len(tokens)-1)
               if tokens[i] in '.?!']

In [25]:
featuresets[100]

({'next-word-capitalized': True,
  'prev-word': 'million',
  'punct': '.',
  'prev-word-is-one-char': False},
 True)

In [27]:
featuresets[2]

({'next-word-capitalized': True,
  'prev-word': 'mr',
  'punct': '.',
  'prev-word-is-one-char': False},
 False)

Now we can train a classifier.

In [29]:
size = int(len(featuresets) * 0.1)
train_set, test_set = featuresets[size:], featuresets[:size]
classifier = nltk.NaiveBayesClassifier.train(train_set)
nltk.classify.accuracy(classifier, test_set)

0.936026936026936


We can use this classifier to perform sentence segmentation as show below:

In [88]:
# The function below has been slightly modified from that in the NLTK book.
# It is now able to segment multiple sentences, in a list of words
def segment_sentences(words, sents=[], start=0):
    for i, word in enumerate(words):
        if word in '.?!' and classifier.classify(punct_features(words, i)) == True:
            sents.append(words[start:i+1])
            start = i+1
    if start < len(words):
        segment_sentences(words[start:], sents, start)
#         sents.append(words[start:])
    return sents

In [89]:
words = tokens[:120]

In [90]:
for sent in segment_sentences(words):
    print(sent)

['.']
['START', 'Pierre', 'Vinken', ',', '61', 'years', 'old', ',', 'will', 'join', 'the', 'board', 'as', 'a', 'nonexecutive', 'director', 'Nov', '.', '29', '.', 'Mr', '.', 'Vinken', 'is', 'chairman', 'of', 'Elsevier', 'N', '.', 'V', '.,', 'the', 'Dutch', 'publishing', 'group', '.']
['.', 'START', 'Rudolph', 'Agnew', ',', '55', 'years', 'old', 'and', 'former', 'chairman', 'of', 'Consolidated', 'Gold', 'Fields', 'PLC', ',', 'was', 'named', 'a', 'nonexecutive', 'director', 'of', 'this', 'British', 'industrial', 'conglomerate', '.', '.', 'START', 'A', 'form', 'of', 'asbestos', 'once', 'used', 'to', 'make', 'Kent', 'cigarette', 'filters', 'has', 'caused', 'a', 'high', 'percentage', 'of', 'cancer', 'deaths', 'among', 'a', 'group', 'of', 'workers', 'exposed', 'to', 'it', 'more', 'than', '30', 'years', 'ago', ',', 'researchers', 'reported', '.']


### Identifying Dialogue Act Types

The NPS Chat Corpus consists of over 10,000 posts from instant messaging sessions. These posts have all been labeled with one of 15 dialogue act types, such as "Statement," "Emotion," "ynQuestion", and "Continuer." 

We can therefore use this data to build a classifier that can identify the dialogue act types for new instant messaging posts.

Let's begin with extracting the basic messaging data:

In [91]:
posts = nltk.corpus.nps_chat.xml_posts()[:10000]

In [94]:
# Defining a feature extractor to check what words a post contains
def dialogue_act_features(post):
    return {'contains({})'.format(word.lower()): True for word in nltk.word_tokenize(post)}

In [95]:
# creating featuresets
featuresets = [(dialogue_act_features(post.text), post.get('class')) for post in posts]

In [96]:
featuresets[0]

({'contains(now)': True,
  'contains(im)': True,
  'contains(left)': True,
  'contains(with)': True,
  'contains(this)': True,
  'contains(gay)': True,
  'contains(name)': True},
 'Statement')

In [97]:
# constructing training and test data
size = int(len(featuresets) * 0.1)
train_set, test_set = featuresets[size:], featuresets[:size]

In [98]:
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.668


### Recognizing Textual Entailment

Recognizing textual entailment (RTE) is the task of determining whether a given piece of text T entails another text called the "hypothesis". 

We can treat RTE as a classification task, in which we try to predict whether the Text, Hypethesis pair is True or False. 

Let's start with building a feature extractor for this task.

In [99]:
def rte_features(rtepair):
    extractor = nltk.RTEFeatureExtractor(rtepair)
    features = {}
#     word_overlap: degree of word overlap between H and T
    features['word_overlap'] = len(extractor.overlap('word'))
#     word_hyp_extra: degree to which there are words in H but not in T
    features['word_hyp_extra'] = len(extractor.hyp_extra('word'))
#     degree of named entity overlap
    features['ne_overlap'] = len(extractor.overlap('ne'))
    features['ne_hyp_extra'] = len(extractor.hyp_extra('ne'))
    return features

In [101]:
# Examining the extracted features of an RTE pair
rtepair = nltk.corpus.rte.pairs(['rte3_dev.xml'])[33]
extractor = nltk.RTEFeatureExtractor(rtepair)
print(extractor.text_words)

{'operation', 'Asia', 'four', 'Organisation', 'together', 'that', 'fight', 'Russia', 'former', 'meeting', 'SCO', 'central', 'association', 'terrorism.', 'fledgling', 'Davudi', 'Soviet', 'China', 'Shanghai', 'republics', 'Parviz', 'binds', 'Co', 'representing', 'at', 'was', 'Iran'}


In [102]:
print(extractor.hyp_words)

{'SCO.', 'member', 'China'}


In [103]:
print(extractor.overlap('word'))

set()


In [104]:
print(extractor.overlap('ne'))

{'China'}


In [105]:
print(extractor.hyp_extra('word'))

{'member'}


Further reading: https://www.nltk.org/book/ch06.html